In [1]:
import os, sys
import pandas as pd
import numpy as np

path = os.getcwd()

In [2]:
FFNAME = "czapla2022"

### Lists and defined functions:

In [3]:
theta_lst = ['tilt','roll','twist','shift','slide','rise']

# --- Lists of dimers, tetramers, and the self-complementary steps ---
BASES     = ['A','C','G','T']
COMP      = {'A':'T', 'T':'A', 'C':'G', 'G':'C','.':'.'}

def dna_seq_complement(sequence_string):
    COMP = {'A':'T', 'T':'A', 'C':'G', 'G':'C','.':'.'}
    STEP = sequence_string[::-1]
    STEP = ''.join([COMP[STEP[i]] for i in range(len(STEP))])
    return STEP

DIMERS    = [b+c for b in BASES for c in BASES]
DIM_MATCH = {'TT':'AA', 'GT':'AC', 'CT':'AG', 'CC':'GG', 'TC':'GA', 'TG':'CA'}
DIMER_LST = ['AT','AC','GC'] + ['AG','GG','GA','AA'] + ['CG','CA','TA']
SCDIM     = []
for DIM in DIMERS:
    if COMP[DIM[1]]+COMP[DIM[0]]==DIM:
        SCDIM.append(DIM)

TET_LST      = [a+b+c+d for a in ['A','C','G','T','.'] for b in BASES for c in BASES for d in ['A','C','G','T','.']]
TETRAMERS    = [i for i in TET_LST if '.' not in i]
TETRAMER_LST = [
    'AAAA','AACA','AAGA','AATA','ACAA','ACGA','AGAA','AGCA','AGGA','ATAA',
    'AAAC','AACC','AAGC','AATC','ACAC','ACGC','AGAC','AGCC','AGGC','ATAC',
    'AAAG','AACG','AAGG','AATG','ACAG','ACGG','AGAG','AGCG','AGGG','ATAG',
    'AAAT','AACT','AAGT','AATT','ACAT','ACGT','AGAT','AGCT','AGGT','ATAT',
    'CAAA','CACA','CAGA','CATA','CCAA','CCGA','CGAA','CGCA','CGGA','CTAA',
    'CAAC','CACC','CAGC','CCAC','CGAC','CGGC','CAAG','CACG','CAGG','CATG',
    'CCAG','CCGG','CGAG','CGCG','CGGG','CTAG','CAAT','CACT','CAGT','CCAT',
    'CGAT','CGGT','GAAA','GACA','GAGA','GATA','GCAA','GCGA','GGAA','GGCA',
    'GGGA','GTAA','GAAC','GACC','GAGC','GATC','GCAC','GCGC','GGAC','GGCC',
    'GGGC','GTAC','GAAG','GACG','GAGG','GATG','GCAG','GCGG','GGAG','GGCG',
    'GGGG','GTAG','GAAT','GACT','GAGT','GCAT','GGAT','GGGT','TAAA','TACA',
    'TAGA','TATA','TCAA','TCGA','TGAA','TGCA','TGGA','TTAA','TAAC','TACC',
    'TAGC','TCAC','TGAC','TGGC','TAAG','TACG','TAGG','TCAG','TGAG','TGGG',
    'TAAT','TACT','TAGT','TCAT','TGAT','TGGT'
]
SCTET        = []
for TET in TETRAMERS:
    if COMP[TET[3]]+COMP[TET[2]]+COMP[TET[1]]+COMP[TET[0]]==TET and '.' not in TET:
        SCTET.append(TET)

In [4]:
def steppars_output(OUTFILENAME, DATAFRAME):
    idx_lst = DATAFRAME.index.tolist()
    outfile=open(OUTFILENAME, "w")    
    for i in range(len(DATAFRAME)):
        STEP=idx_lst[i]
        if '.' in STEP:
            STEP=STEP.replace('.','x')
        DATA = tuple( map( lambda x: isinstance(x, float) and round(x, 3) or x, DATAFRAME.loc[idx_lst[i]]) )
        LINE = "{0}".format(STEP)+'={'+"{0}, {1}, {2}, {3}, {4}, {5}".format(*DATA)+'}\n'
        outfile.write(LINE)
        del LINE, DATA, STEP
    outfile.close()
    return 

def forcecons_output(OUTFILENAME, DATAFRAME):
    idx_lst = DATAFRAME.index.tolist()
    outfile=open(OUTFILENAME, "w")    
    for i in range(len(DATAFRAME)):
        STEP=idx_lst[i]
        if '.' in STEP:
            STEP=STEP.replace('.','x')
        DATA = tuple( map( lambda x: isinstance(x, float) and round(x, 5) or x, DATAFRAME.loc[idx_lst[i]]) )
        LINE = "{0}".format(STEP)+'={'+"{0}, {1}, {2}, {3}, {4}, {5}, {6}, {7}, {8}, {9}, {10}, {11}, {12}, {13}, {14}, {15}, {16}, {17}, {18}, {19}, {20}, {21}, {22}, {23}, {24}, {25}, {26}, {27}, {28}, {29}, {30}, {31}, {32}, {33}, {34}, {35}".format(*DATA)+'}\n'
        outfile.write(LINE)
        del LINE, DATA, STEP
    outfile.close()
    return

------------
## Standard Forcefield

### Structural Space, dimeric

In [ ]:
dataset = pd.read_csv(FFNAME+"_3sig_tet", index_col=0)

dataset.head(5)

In [ ]:
# THIS is for STRUCTURAL space.

ffdf_1 = pd.DataFrame(columns=[i for i in theta_lst],  index=DIMERS)
for DIMER in DIMERS:
    for THETA1 in theta_lst:
        ffdf_1.at[DIMER, THETA1] = round(dataset.loc[(dataset.step_dimer==DIMER)][THETA1].mean(), 3)    
ffdf_1 = ffdf_1.astype(float)


In [ ]:
ffdf_1

In [ ]:
ffdf_2 = ffdf_1.copy()

for t1 in theta_lst:
    for t2 in theta_lst:
        dataset[t1+'.'+t2] = dataset[t1] * dataset[t2]
        ffdf_2["<"+t1+'>.<'+t2+">"] = ffdf_2[t1]*ffdf_2[t2]

for DIMER in DIMERS:
    for t1 in theta_lst:
        for t2 in theta_lst:
            ffdf_2.at[DIMER, "<"+t1+'.'+t2+">"]   = round( dataset.loc[(dataset.step_dimer==DIMER)][t1+'.'+t2].mean(), 5)

In [ ]:
ffdf_2

In [ ]:
covdf  = pd.DataFrame(index=ffdf_2.index)
for t1 in theta_lst:
    for t2 in theta_lst:
        covdf[t1+'.'+t2] = round( ffdf_2["<"+t1+'.'+t2+">"] - ffdf_2["<"+t1+'>.<'+t2+">"], 6)

fcondf = pd.DataFrame(columns=covdf.columns, index=ffdf_1.index)

for DIMER in DIMERS:
    mat_covar = covdf.loc[DIMER].to_numpy()
    mat_covar = np.reshape(mat_covar, (6,6))
    mat_inv   = np.linalg.inv(mat_covar)
    mat_inv   = np.reshape(mat_inv,  (1, 36))
    fcondf.loc[DIMER] = mat_inv
    del mat_covar, mat_inv
    
#for DIMER in DIMERS:
#    if DIMER not in DIMER_LST:
#        fcondf.loc[DIMER] = fcondf.loc[DIM_MATCH[DIMER]] 
    
fcondf=fcondf.astype(float).round(6)


In [ ]:
for DIMER in DIMERS:
    mat = covdf.loc[DIMER].to_numpy()
    mat = np.reshape(mat, (6,6))
    if (mat==mat.T).all():
        print(DIMER+"\t", 
              str(round( np.sqrt( np.prod( np.linalg.eig(np.reshape(mat, (6,6)))[0] ) ), 2))+"\t",
              str(round( np.prod( np.linalg.eig(np.reshape(mat, (6,6)))[0] ), 2))+"\t",
              np.linalg.eig(np.reshape(mat, (6,6)))[0])
    del mat

In [ ]:
steppars_output("StepParameters_"+FFNAME+"_dim.txt", ffdf_1[theta_lst])

covdf.to_csv(FFNAME+"_dim_parameter-covariance-data")
fcondf.to_csv(FFNAME+"_dim_force-constants-data")
forcecons_output("ForceConstants_"+FFNAME+"_dim.txt", fcondf)
del covdf, fcondf

del ffdf_1, ffdf_2

del dataset


### Structural Space, tetrameric

In [ ]:
dataset = pd.read_csv(FFNAME+"_3sig_tet", index_col=0)

dataset.head(5)

In [ ]:
# Recall: The force field object will have 400 entries: 4^4 4-base tetramers, 2* 3^4 terminal trimers, and 2^4 terminal dimers

ffdf_1 = pd.DataFrame(columns=[i for i in theta_lst],  index=DIMERS)

for DIMER in DIMERS:
    for THETA1 in theta_lst:
        ffdf_1.at[DIMER, THETA1] = round(dataset.loc[(dataset.step_dimer==DIMER)][THETA1].mean(), 3)

ffdf_1 = ffdf_1.astype(float)


ffdf_2 = pd.DataFrame(columns=[i for i in theta_lst],  index=TET_LST)
for TETRAMER in TET_LST:
    
    if TETRAMER in TETRAMERS:
        for THETA1 in theta_lst:
            ffdf_2.at[TETRAMER, THETA1] = round(dataset.loc[(dataset.step_tetramer==TETRAMER)][THETA1].mean(), 3)

    else:
        DIMER = TETRAMER[1:3]
        for THETA1 in theta_lst:
            ffdf_2.at[TETRAMER, THETA1] = ffdf_1.at[DIMER, THETA1]
        del DIMER

ffdf_2 = ffdf_2.astype(float)
steppars_output("StepParameters_"+FFNAME+"_tet.txt", ffdf_2[theta_lst])
del ffdf_1


In [ ]:
for t1 in theta_lst:
    for t2 in theta_lst:
        dataset[t1+'.'+t2] = dataset[t1] * dataset[t2]


In [ ]:
ffdf = ffdf_2.copy()[~ffdf_2.index.str.contains('\.')]

for t1 in theta_lst:
    for t2 in theta_lst:
        ffdf["<"+t1+'>.<'+t2+">"] = ffdf[t1]*ffdf[t2]

for TETRAMER in TETRAMERS:
    for t1 in theta_lst:
        for t2 in theta_lst:
            ffdf.at[TETRAMER, "<"+t1+'.'+t2+">"]   = round( dataset.loc[(dataset.step_tetramer==TETRAMER)][t1+'.'+t2].mean(), 5)


In [ ]:
covdf = pd.DataFrame(index=ffdf.index)
for t1 in theta_lst:
    for t2 in theta_lst:
        covdf[t1+'.'+t2] = round( ffdf["<"+t1+'.'+t2+">"] - ffdf["<"+t1+'>.<'+t2+">"], 6)


fcondf_TET = pd.DataFrame(columns=covdf.columns, index=TETRAMERS)

for TETRAMER in TETRAMERS:
    mat_covar = covdf.loc[TETRAMER].to_numpy()
    mat_covar = np.reshape(mat_covar, (6,6))
    mat_inv   = np.linalg.inv(mat_covar)
    mat_inv   = np.reshape(mat_inv,  (1, 36))
    fcondf_TET.loc[TETRAMER] = mat_inv
    del mat_covar, mat_inv
    
fcondf_TET=fcondf_TET.astype(float).round(6)

fcondf_TET

In [ ]:
tet_fc_df=pd.DataFrame(columns=fcondf_TET.columns, index=TET_LST)

fcondf = pd.read_csv(FFNAME+"_dim_force-constants-data", index_col=0)


for IDX in tet_fc_df.index:
    if IDX in fcondf_TET.index:
        tet_fc_df.loc[IDX] = fcondf_TET.loc[IDX]
    else:
        DIM = IDX[1:3]
        tet_fc_df.loc[IDX] = fcondf.loc[DIM]
        del DIM
        
tet_fc_df

In [ ]:
tet_fc_df.loc[['AAAA','TTTT']]

In [ ]:
covdf.to_csv(FFNAME+"_tet_parameter-covariance-data")
fcondf_TET.to_csv(FFNAME+"_tet_force-constants-data")
forcecons_output("ForceConstants_"+FFNAME+"_tet.txt", tet_fc_df)
del covdf, fcondf_TET, fcondf
del tet_fc_df

del ffdf_2

del dataset

--------

## Try: PDB-series Data

### Step Paramaeters + Force Constants

#### Dimeric 

In [5]:
set_path = path+'/pdb_time-series-data'

for YEAR in [2000+2*i for i in range(0, 12)]:

    dfd1    = pd.read_csv(set_path+"/czapla2022_pdb-series_year-series_"+str(YEAR)+"_data",index_col=0)
    
    dataset = dfd1.copy()[['step_dimer','tilt','roll','twist','shift','slide','rise']]
    ffdf_1 = pd.DataFrame(columns=[i for i in theta_lst],  index=DIMERS)
    for DIMER in DIMERS:
        for THETA1 in theta_lst:
            ffdf_1.at[DIMER, THETA1] = round(dataset.loc[(dataset.step_dimer==DIMER)][THETA1].mean(), 3)
    ffdf_1 = ffdf_1.astype(float)
    
    
    ffdf_2 = ffdf_1.copy()
    for t1 in theta_lst:
        for t2 in theta_lst:
            dataset[t1+'.'+t2] = dataset[t1] * dataset[t2]
            ffdf_2["<"+t1+'>.<'+t2+">"] = ffdf_2[t1]*ffdf_2[t2]
    for DIMER in DIMERS:
        for t1 in theta_lst:
            for t2 in theta_lst:
                ffdf_2.at[DIMER, "<"+t1+'.'+t2+">"]   = round( dataset.loc[(dataset.step_dimer==DIMER)][t1+'.'+t2].mean(), 5)
    
    covdf = pd.DataFrame(index=ffdf_2.index)
    for t1 in theta_lst:
        for t2 in theta_lst:
            covdf[t1+'.'+t2] = round( ffdf_2["<"+t1+'.'+t2+">"] - ffdf_2["<"+t1+'>.<'+t2+">"], 6)
    
    fcondf = pd.DataFrame(columns=covdf.columns, index=ffdf_1.index)
    for DIMER in DIMERS:
        mat_covar = covdf.loc[DIMER].to_numpy()
        mat_covar = np.reshape(mat_covar, (6,6))
        mat_inv   = np.linalg.inv(mat_covar)
        mat_inv   = np.reshape(mat_inv,  (1, 36))
        fcondf.loc[DIMER] = mat_inv
        del mat_covar, mat_inv
    fcondf=fcondf.astype(float).round(6)    
    
    
    covdf.to_csv(set_path+"/"+FFNAME+"_pdb-series_year-series_"+str(YEAR)+"_dim_par-covariance")
    fcondf.to_csv(set_path+"/"+FFNAME+"_pdb-series_year-series_"+str(YEAR)+"_dim_force-constants")
    
    steppars_output(set_path+"/StepParameters_"+FFNAME+"_"+str(YEAR)+"_dim.txt", ffdf_1[theta_lst])
    forcecons_output(set_path+"/ForceConstants_"+FFNAME+"_"+str(YEAR)+"_dim.txt", fcondf)
    
    del dfd1, dataset, ffdf_1, ffdf_2, covdf, fcondf

del set_path

In [ ]:
set_path = path+'/pdb_res-series-data'

for RES in [1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 6.0, 10.0]:

    dfd1    = pd.read_csv(set_path+"/czapla2022_pdb-series_res-series_"+str(int(RES*10)).zfill(3)+"_data",index_col=0)
    dataset = dfd1.copy()[['step_dimer','tilt','roll','twist','shift','slide','rise']]
    ffdf_1 = pd.DataFrame(columns=[i for i in theta_lst],  index=DIMERS)
    for DIMER in DIMERS:
        for THETA1 in theta_lst:
            ffdf_1.at[DIMER, THETA1] = round(dataset.loc[(dataset.step_dimer==DIMER)][THETA1].mean(), 3)
    ffdf_1 = ffdf_1.astype(float)
    
    
    ffdf_2 = ffdf_1.copy()
    for t1 in theta_lst:
        for t2 in theta_lst:
            dataset[t1+'.'+t2] = dataset[t1] * dataset[t2]
            ffdf_2["<"+t1+'>.<'+t2+">"] = ffdf_2[t1]*ffdf_2[t2]
    for DIMER in DIMERS:
        for t1 in theta_lst:
            for t2 in theta_lst:
                ffdf_2.at[DIMER, "<"+t1+'.'+t2+">"]   = round( dataset.loc[(dataset.step_dimer==DIMER)][t1+'.'+t2].mean(), 5)
    
    covdf = pd.DataFrame(index=ffdf_2.index)
    for t1 in theta_lst:
        for t2 in theta_lst:
            covdf[t1+'.'+t2] = round( ffdf_2["<"+t1+'.'+t2+">"] - ffdf_2["<"+t1+'>.<'+t2+">"], 6)
    
    fcondf = pd.DataFrame(columns=covdf.columns, index=ffdf_1.index)
    for DIMER in DIMERS:
        mat_covar = covdf.loc[DIMER].to_numpy()
        mat_covar = np.reshape(mat_covar, (6,6))
        mat_inv   = np.linalg.inv(mat_covar)
        mat_inv   = np.reshape(mat_inv,  (1, 36))
        fcondf.loc[DIMER] = mat_inv
        del mat_covar, mat_inv
    fcondf=fcondf.astype(float).round(6)    
    
    
    covdf.to_csv(set_path+"/"+FFNAME+"_pdb-series_res-series_"+str(int(RES*10)).zfill(3)+"_dim_par-covariance")
    fcondf.to_csv(set_path+"/"+FFNAME+"_pdb-series_res-series_"+str(int(RES*10)).zfill(3)+"_dim_force-constants")
    
    steppars_output(set_path+"/StepParameters_"+FFNAME+"_res"+str(int(RES*10)).zfill(3)+"_dim.txt", ffdf_1[theta_lst])
    forcecons_output(set_path+"/ForceConstants_"+FFNAME+"_res"+str(int(RES*10)).zfill(3)+"_dim.txt", fcondf)
    
    del dfd1, dataset, ffdf_1, ffdf_2, covdf, fcondf

del set_path

#### Tetrameric

In [9]:
set_path = path+'/pdb_time-series-data'

for YEAR in [2000+2*i for i in range(0, 12)]:

    dfd1    = pd.read_csv(set_path+"/czapla2022_pdb-series_year-series_"+str(YEAR)+"_data",index_col=0)
    dataset = dfd1.copy()[['step_dimer','step_tetramer','tilt','roll','twist','shift','slide','rise']]
    
    ffdf_1 = pd.DataFrame(columns=[i for i in theta_lst],  index=DIMERS)
    for DIMER in DIMERS:
        for THETA1 in theta_lst:
            ffdf_1.at[DIMER, THETA1] = round(dataset.loc[(dataset.step_dimer==DIMER)][THETA1].mean(), 3)
    ffdf_1 = ffdf_1.astype(float)
    
    ffdf_2 = pd.DataFrame(columns=[i for i in theta_lst],  index=TET_LST)
    for TETRAMER in TET_LST:
        if TETRAMER in TETRAMERS:
            for THETA1 in theta_lst:
                ffdf_2.at[TETRAMER, THETA1] = round(dataset.loc[(dataset.step_tetramer==TETRAMER)][THETA1].mean(), 3)
        else:
            DIMER = TETRAMER[1:3]
            for THETA1 in theta_lst:
                ffdf_2.at[TETRAMER, THETA1] = ffdf_1.at[DIMER, THETA1]
            del DIMER
    ffdf_2 = ffdf_2.astype(float)   
    
    for t1 in theta_lst:
        for t2 in theta_lst:
            dataset[t1+'.'+t2] = dataset[t1] * dataset[t2]
            
    ffdf = ffdf_2.copy()[~ffdf_2.index.str.contains('\.')]    
    for t1 in theta_lst:
        for t2 in theta_lst:
            ffdf["<"+t1+'>.<'+t2+">"] = ffdf[t1]*ffdf[t2]

    for TETRAMER in TETRAMERS:
        for t1 in theta_lst:
            for t2 in theta_lst:
                ffdf.at[TETRAMER, "<"+t1+'.'+t2+">"]   = round( dataset.loc[(dataset.step_tetramer==TETRAMER)][t1+'.'+t2].mean(), 5)
    
    covdf = pd.DataFrame(index=ffdf_2.index)
    for t1 in theta_lst:
        for t2 in theta_lst:
            covdf[t1+'.'+t2] = round( ffdf["<"+t1+'.'+t2+">"] - ffdf["<"+t1+'>.<'+t2+">"], 6)
    
    fcondf_TET = pd.DataFrame(columns=covdf.columns, index=TETRAMERS)
    for TETRAMER in TETRAMERS:
        mat_covar = covdf.loc[TETRAMER].to_numpy()
        mat_covar = np.reshape(mat_covar, (6,6))
        mat_inv   = np.linalg.inv(mat_covar)
        mat_inv   = np.reshape(mat_inv,  (1, 36))
        fcondf_TET.loc[TETRAMER] = mat_inv
        del mat_covar, mat_inv
    fcondf_TET=fcondf_TET.astype(float).round(6)  
    tet_fc_df=pd.DataFrame(columns=fcondf_TET.columns, index=TET_LST)
    fcondf = pd.read_csv(set_path+"/czapla2022_pdb-series_year-series_"+str(YEAR)+"_dim_force-constants",index_col=0)
    for IDX in tet_fc_df.index:
        if IDX in fcondf_TET.index:
            tet_fc_df.loc[IDX] = fcondf_TET.loc[IDX]
        else:
            DIM = IDX[1:3]
            tet_fc_df.loc[IDX] = fcondf.loc[DIM]
            del DIM
    
    covdf.to_csv(set_path+"/"+FFNAME+"_pdb-series_year-series_"+str(YEAR)+"_tet_par-covariance")
    fcondf.to_csv(set_path+"/"+FFNAME+"_pdb-series_year-series_"+str(YEAR)+"_tet_force-constants")
    
    steppars_output(set_path+"/StepParameters_"+FFNAME+"_"+str(YEAR)+"_tet.txt", ffdf_2[theta_lst])
    forcecons_output(set_path+"/ForceConstants_"+FFNAME+"_"+str(YEAR)+"_tet.txt", tet_fc_df)
    
    del dfd1, dataset, ffdf_1, ffdf_2, covdf, fcondf, fcondf_TET, tet_fc_df

del set_path

In [12]:
set_path = path+'/pdb_res-series-data'

for RES in [1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 6.0, 10.0]:

    dfd1    = pd.read_csv(set_path+"/czapla2022_pdb-series_res-series_"+str(int(RES*10)).zfill(3)+"_data",index_col=0)
    dataset = dfd1.copy()[['step_dimer','step_tetramer','tilt','roll','twist','shift','slide','rise']]
    
    ffdf_1 = pd.DataFrame(columns=[i for i in theta_lst],  index=DIMERS)
    for DIMER in DIMERS:
        for THETA1 in theta_lst:
            ffdf_1.at[DIMER, THETA1] = round(dataset.loc[(dataset.step_dimer==DIMER)][THETA1].mean(), 3)
    ffdf_1 = ffdf_1.astype(float)
    
    ffdf_2 = pd.DataFrame(columns=[i for i in theta_lst],  index=TET_LST)
    for TETRAMER in TET_LST:
        if TETRAMER in TETRAMERS:
            for THETA1 in theta_lst:
                ffdf_2.at[TETRAMER, THETA1] = round(dataset.loc[(dataset.step_tetramer==TETRAMER)][THETA1].mean(), 3)
        else:
            DIMER = TETRAMER[1:3]
            for THETA1 in theta_lst:
                ffdf_2.at[TETRAMER, THETA1] = ffdf_1.at[DIMER, THETA1]
            del DIMER
    ffdf_2 = ffdf_2.astype(float)   
    
    for t1 in theta_lst:
        for t2 in theta_lst:
            dataset[t1+'.'+t2] = dataset[t1] * dataset[t2]
            
    ffdf = ffdf_2.copy()[~ffdf_2.index.str.contains('\.')]    
    for t1 in theta_lst:
        for t2 in theta_lst:
            ffdf["<"+t1+'>.<'+t2+">"] = ffdf[t1]*ffdf[t2]

    for TETRAMER in TETRAMERS:
        for t1 in theta_lst:
            for t2 in theta_lst:
                ffdf.at[TETRAMER, "<"+t1+'.'+t2+">"]   = round( dataset.loc[(dataset.step_tetramer==TETRAMER)][t1+'.'+t2].mean(), 5)
    
    covdf = pd.DataFrame(index=ffdf_2.index)
    for t1 in theta_lst:
        for t2 in theta_lst:
            covdf[t1+'.'+t2] = round( ffdf["<"+t1+'.'+t2+">"] - ffdf["<"+t1+'>.<'+t2+">"], 6)
    
    fcondf_TET = pd.DataFrame(columns=covdf.columns, index=TETRAMERS)
    for TETRAMER in TETRAMERS:
        mat_covar = covdf.loc[TETRAMER].to_numpy()
        mat_covar = np.reshape(mat_covar, (6,6))
        mat_inv   = np.linalg.inv(mat_covar)
        mat_inv   = np.reshape(mat_inv,  (1, 36))
        fcondf_TET.loc[TETRAMER] = mat_inv
        del mat_covar, mat_inv
    fcondf_TET=fcondf_TET.astype(float).round(6)  
    tet_fc_df=pd.DataFrame(columns=fcondf_TET.columns, index=TET_LST)
    fcondf = pd.read_csv(set_path+"/czapla2022_pdb-series_res-series_"+str(int(RES*10)).zfill(3)+"_dim_force-constants",index_col=0)
    for IDX in tet_fc_df.index:
        if IDX in fcondf_TET.index:
            tet_fc_df.loc[IDX] = fcondf_TET.loc[IDX]
        else:
            DIM = IDX[1:3]
            tet_fc_df.loc[IDX] = fcondf.loc[DIM]
            del DIM
            
            
    covdf.to_csv(set_path+"/"+FFNAME+"_pdb-series_res-series_"+str(int(RES*10)).zfill(3)+"_tet_par-covariance")
    fcondf.to_csv(set_path+"/"+FFNAME+"_pdb-series_res-series_"+str(int(RES*10)).zfill(3)+"_tet_force-constants")
    
    steppars_output(set_path+"/StepParameters_"+FFNAME+"_res"+str(int(RES*10)).zfill(3)+"_tet.txt", ffdf_2[theta_lst])
    forcecons_output(set_path+"/ForceConstants_"+FFNAME+"_res"+str(int(RES*10)).zfill(3)+"_tet.txt", tet_fc_df)
    
    del dfd1, dataset, ffdf_1, ffdf_2, covdf, fcondf

del set_path